In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

In [2]:
train_iter = []
train_df =  pd.read_csv('training_dataset.csv')
for i, row in tqdm(train_df.iterrows()):
    train_iter.append((row.label, str(row.text)))

27514it [00:03, 8807.31it/s]


In [3]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

tokenizer = get_tokenizer('spacy')
train_iter = train_iter
counter = Counter()
for (label, line) in tqdm(train_iter):
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

100%|██████████| 27514/27514 [00:01<00:00, 14939.11it/s]


In [4]:
def save_vocab(vocab, path):
    import pickle
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()
save_vocab(vocab, 'vocab.pkl')

In [5]:
[vocab[token] for token in ['here', 'is', 'an', 'example']]

[150, 5, 59, 2276]

In [6]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x)

In [7]:
text_pipeline('here is the an example')

[150, 5, 4, 59, 2276]

In [8]:
label_pipeline('0')

0

In [9]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = train_iter
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [10]:
from torch import nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim, 150)
        self.fc2 = nn.Linear(150, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        return F.softmax(self.fc2(x))

In [11]:
train_iter = train_iter
num_class = 2
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [12]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [13]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = train_test_split(train_iter, test_size=0.2)
train_dataset = list(train_iter)
test_dataset = list(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

<ipython-input-10-616a7dac7069>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.fc2(x))


-----------------------------------------------------------
| end of epoch   1 | time:  2.22s | valid accuracy    0.992 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  2.18s | valid accuracy    0.993 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  2.20s | valid accuracy    0.993 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  2.19s | valid accuracy    0.993 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  2.20s | valid accuracy    0.993 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  2.21s |

In [14]:
torch.save(model.state_dict(), './model.pth')

In [15]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.


<ipython-input-10-616a7dac7069>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.fc2(x))


test accuracy    0.992


In [16]:
print(train_iter[8])
# tokenizer(BeautifulSoup(train_iter[1][1], 'html.parser').get_text())

(1, 'vietnam.')


In [17]:
def text_predictor(text):
    processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
    offset = torch.tensor([0], dtype=torch.int64)
    predited_label = model(processed_text, offset)
    output = np.argmax(predited_label.detach().numpy())
    print(output)
    return "ANSWER" if output == 1   else "QUESTION"

In [18]:
text = "my javascript code is breaking at line number 1"
text_predictor(text)

0


<ipython-input-10-616a7dac7069>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.fc2(x))


'QUESTION'